# Projeto de Automação com API's - Telegram integrado ao Goolge Sheets

## 1\. Contexto

Este é um projeto que utiliza a biblioteca telebot do Python para criar um bot no Telegram que recebe mensagens contendo informações de itens, quantidades e preços, e os armazena em uma planilha do Google Sheets.

As principais funcionalidades do projeto incluem:

- Receber mensagens no formato "item/quantidade/preço" e salvá-las em uma planilha do Google Sheets
- Extrair informações de data atual e conteúdo da mensagem recebida
- Autenticar com as credenciais da API do Google Sheets e usar a biblioteca googleapiclient para acessar e manipular planilhas
- Tratar erros de autenticação e de inserção de valores na planilha

No geral, este projeto pode ser útil para empresas ou indivíduos que desejam armazenar informações de forma automatizada e organizada, especialmente para controle de estoque ou finanças. O uso do Telegram como plataforma de comunicação permite que os usuários enviem informações rapidamente e com facilidade, enquanto a integração com o Google Sheets permite o armazenamento e organização dessas informações em uma planilha acessível em qualquer lugar com conexão à internet.

## 2\. Preparação

####2.1.BotFather

Para criar um bot no Telegram e obter uma chave de API, siga os seguintes passos:

- Abra o aplicativo do Telegram e procure pelo bot "@BotFather".
- Clique em "Iniciar" para iniciar uma conversa com o BotFather.
- Envie a mensagem "/newbot" para o BotFather e siga as instruções para criar um novo bot. Você precisará fornecer um nome e um username para o seu bot.
- Após criar o bot, o BotFather irá fornecer uma chave de API que você deve salvar em um local seguro. Essa chave é necessária para autenticar o seu bot sempre que ele se comunicar com o servidor do Telegram.

Pronto! Agora você pode usar essa chave de API para autenticar o seu bot em qualquer aplicativo ou script que você estiver desenvolvendo para interagir com o Telegram.





 > Acesse pelo Telegram: [BotFather](https://telegram.me/BotFather).

####2.2.Google Cloud Platform e API do Google Sheets

Criando um Projeto no Google Cloud Platform
Para podermos acessar a API do Google Sheets, precisamos criar um projeto no Google Cloud Platform e habilitar a API do Google Sheets. Siga as instruções abaixo:

- Faça login no Google Cloud Console.
- Crie um novo projeto clicando no botão "Select a project" no canto superior esquerdo da página e, em seguida, clicando em "New Project".
- Digite um nome para o projeto e clique em "Create".
- Selecione o projeto recém-criado no canto superior esquerdo da página.
- Habilitar a API do Google Sheets. No painel de navegação à esquerda, selecione "APIs & Services" e, em seguida, "Library". Na barra de pesquisa, digite "Google Sheets API" e clique em "Enable".
- Criar credenciais. No painel de navegação à esquerda, selecione "APIs & Services" e, em seguida, "Credentials". - Clique em "Create Credentials" e, em seguida, selecione "Service account key". Preencha as informações solicitadas e clique em "Create". Isso irá baixar um arquivo JSON contendo as credenciais de serviço.
- Compartilhar a planilha. Abra a planilha que você deseja usar e compartilhe-a com o endereço de e-mail do serviço criado nas etapas anteriores.

 > Tutorial de como criar  a credencial e salva-la em arquivo Json: [YouTube](https://www.youtube.com/watch?v=llQb7w1zzd8).

## 3\.Bibliotecas 

As bibliotecas utilizadas são:

- telebot: biblioteca para trabalhar com a API do Telegram.
- ServiceAccountCredentials e build da biblioteca googleapiclient.discovery: biblioteca para trabalhar com a API do Google Sheets.
- datetime: biblioteca para trabalhar com datas e horários.

In [ ]:
pip install pyTelegramBotAPI

In [ ]:
import telebot
import datetime
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

## 4\.Execução

In [ ]:
# Definindo as credenciais do bot do Telegram e das APIs do Google Sheets
telegram_token = 'BOT_TOKEN'
google_credentials = ServiceAccountCredentials.from_json_keyfile_name('credenciais.json', ['https://www.googleapis.com/auth/spreadsheets'])
google_sheets_id = 'ID_SHEETS'

In [ ]:
# Definindo a função para lidar com as mensagens de adição de itens ao estoque
def handle_message(message): 
    chat_id = message.chat.id
    text = message.text
    # Separando a mensagem em uma lista de itens
    items = text.split(',') 
    
    items = message.text.split('/')
    if len(items) < 3:
        bot.reply_to(message, 'Por favor, forneça uma mensagem no formato "item/quantidade/preço".')
        return

A função `handle_message(message)` é a responsável por lidar com as mensagens enviadas pelo usuário para o bot. Essa função recebe como parâmetro uma mensagem, que contém informações como o texto enviado pelo usuário e o ID do chat em que a mensagem foi enviada.

As primeiras linhas da função fazem o processamento da mensagem enviada pelo usuário. Primeiramente, é feita a separação das informações enviadas pelo usuário, utilizando o caractere `","` como separador. Em seguida, é feita uma verificação se o formato da mensagem está correto, ou seja, se ela contém as informações `"item/quantidade/preço"`. Caso a mensagem não esteja nesse formato, uma mensagem de erro é enviada para o usuário.

In [ ]:
    # Obter a data atual
    current_time = datetime.datetime.now().strftime("%Y-%m-%d")

Se a mensagem estiver no formato correto, as informações são extraídas e armazenadas em variáveis separadas. A variável `current_time` é criada para armazenar a data atual.

In [ ]:
    # Extrair os valores do item, quantidade e preço da mensagem
    description = items[0]
    quantity = items[1]
    value = items[2]

try:
        service = build('sheets', 'v4', credentials=google_credentials)
        sheet = service.spreadsheets()

        # Criar uma lista com os valores do item, quantidade, preço e data atual
        row = [current_time, description, quantity, value]

        # Inserir a lista de valores na planilha na primeira linha
        result = sheet.values().append(
            spreadsheetId=google_sheets_id,
            range='Sheet1!A1',
            valueInputOption='USER_ENTERED',
            insertDataOption='INSERT_ROWS',
            body={'values': [row]}
        ).execute()

        bot.send_message(chat_id, 'Mensagem salva com sucesso!')
    except HttpError as error:
        bot.send_message(chat_id, f'Ocorreu um erro ao salvar a mensagem: {error}')

Em seguida, a função tenta se conectar à API do Google Sheets para salvar as informações na planilha especificada. Para isso, são utilizadas as credenciais do Google Sheets e o identificador da planilha. É criada uma mensagem que contém as informações enviadas pelo usuário e é construído um objeto body com as informações da mensagem para serem inseridas na planilha.

O método `values().append()` é chamado para adicionar as informações na planilha, utilizando as opções de inserção de dados `"USER_ENTERED"` e `"INSERT_ROWS"`. O resultado dessa operação é armazenado na variável result.

Se ocorrer algum erro ao tentar salvar as informações na planilha, uma mensagem de erro é enviada para o usuário.

In [ ]:
# Iniciar o bot com o token
bot = telebot.TeleBot(telegram_token) 

@bot.message_handler(func=lambda message: True)
def handle_all_messages(message):
    handle_message(message)

bot.polling()

Por fim, a função `bot.polling()` é executada em um loop infinito para que o bot fique sempre escutando por novas mensagens enviadas pelos usuários. Qualquer exceção que ocorrer durante a execução é ignorada, garantindo que o bot continue funcionando mesmo em caso de erros.